# io

> Just some helper functions to download datasets etc.

In [ ]:
#| default_exp io

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import httpx
from pathlib import Path

import rich.progress

In [ ]:
#| export

def download_from_url(
    url: str, # url to download
    dst: Path, # the path of the destination file
    chunk_size=2048 # optional chunk_size for response.iter_bytes
) -> int:  # the file size
    if dst.exists():
        file_size = dst.stat().st_size
        if file_size > 0:
            # return early if file was already downloaded
            return file_size
        
    with dst.open("ab") as download_file:
        with httpx.stream("GET", url) as response:
            total = int(response.headers["Content-Length"])
    
            with rich.progress.Progress(
                "[progress.percentage]{task.percentage:>3.0f}%",
                rich.progress.BarColumn(bar_width=None),
                rich.progress.DownloadColumn(),
                rich.progress.TransferSpeedColumn(),
                refresh_per_second=5,
            ) as progress:
                download_task = progress.add_task("Download", total=total)
                for chunk in response.iter_bytes(chunk_size=chunk_size):
                    download_file.write(chunk)
                    progress.update(download_task, completed=response.num_bytes_downloaded, refresh=True)
    return response.num_bytes_downloaded

In [ ]:
#|eval: false

archive_name = "reuters21578.tar.gz"
training_data_url = "http://www.daviddlewis.com/resources/testcollections/reuters21578/{}".format(archive_name)
data_root = Path.cwd().parent / "data"
training_data_path = data_root / archive_name
data_size = download_from_url(training_data_url, training_data_path)
print(data_size)

NameError: name 'Path' is not defined

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()